In [1]:
# Import libraries
import pandas as pd
# import dask.dataframe as pd
import numpy as np
import lightgbm as lgb

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_absolute_error

from utilities import data_basic_utility as databasic
from utilities import dataframe_utility as dfutil
import features_utility as featutil

## File Details - Light GBM Regression All cols inc nlp

This is a Candidate for being used in an Ensemble 2. 
Characteristicts:
* Light GBM Regression Algorithm
* All columns, including Review Counts and NLP doc vecs
* Uses the full files outputted from A3_130
* Todo: use optimised parameters for Light GBM Regression


In [2]:
filePrefix = "A3_140_lgbm_regression_inc_nlp"
baseDataDir = "C:/Development/Data/COSC2670/Assignment3/A3data/"
subrunDir = "subruns/"
writeSubRunFile = False
seed = databasic.get_random_seed()

In [3]:

trainFilePath = baseDataDir + 'train_features_preprocessed.csv'
valiFilePath = baseDataDir + 'vali_features_preprocessed.csv'
testFilePath = baseDataDir + 'test_features_preprocessed.csv'

In [4]:
# RowID  BeerID  ReviewerID  BeerName  BeerType  Label
# df_train = pd.read_csv(baseDataDir + 'train_500k.tsv',sep='\t',
df_train = pd.read_csv(trainFilePath)
df_train.head(10)

,RowID,BeerID,ReviewerID,rating,BeerType_Altbier,BeerType_AmericanAdjunctLager,BeerType_AmericanAmberRedAle,BeerType_AmericanAmberRedLager,BeerType_AmericanBarleywine,BeerType_AmericanBlackAle,...,Lemmatized_DocVec_199,ReviewerReviewCount,BeerReviewCount,ABV,DayofWeek,DayofMonth,Month,Year,TimeOfDay,Birthday
0,19,12300,10635,4.0,0,0,0,0,0,0,...,-0.022973,-0.457702,-0.851519,0.133882,-1.443403,0.828495,-0.454962,1.128552,0.935103,-0.748571
1,21,12300,6547,4.5,0,0,0,0,0,0,...,-0.038961,-0.893674,-0.851519,0.133882,-1.443403,0.033726,-0.454962,1.128552,-2.042387,-0.748571
2,23,12300,9789,4.5,0,0,0,0,0,0,...,-0.028913,-0.540307,-0.851519,0.133882,1.502403,-0.647505,-0.738940,1.128552,0.280057,-0.748571
3,24,12300,7372,5.0,0,0,0,0,0,0,...,-0.034556,0.074644,-0.851519,0.133882,-0.461468,1.623264,-1.022918,1.128552,0.643947,-0.748571
4,25,12300,1302,4.5,0,0,0,0,0,0,...,-0.024277,0.230676,-0.851519,0.133882,0.029500,0.942034,-1.022918,1.128552,0.640656,0.891660
5,26,12300,704,4.5,0,0,0,0,0,0,...,-0.023485,0.471608,-0.851519,0.133882,-0.461468,0.033726,-1.022918,1.128552,0.230800,1.157644
6,29,12300,1747,5.0,0,0,0,0,0,0,...,-0.011979,0.145776,-0.851519,0.133882,0.029500,-1.555812,1.532884,0.714996,0.378846,-0.748571
7,31,12300,9368,4.5,0,0,0,0,0,0,...,-0.021046,-0.804185,-0.851519,0.133882,1.502403,-0.647505,0.964928,0.714996,0.586462,-0.748571
8,32,12300,2568,4.0,0,0,0,0,0,0,...,-0.001757,-0.409515,-0.851519,0.133882,-1.443403,1.282649,0.680950,0.714996,-0.186022,-0.748571
9,33,12300,6838,4.0,0,0,0,0,0,0,...,-0.029366,-0.664215,-0.851519,0.133882,1.011435,-0.533966,0.680950,0.714996,-1.681295,1.379296


In [5]:

# df_vali = pd.read_csv(baseDataDir + 'vali_500k.tsv',sep='\t',
df_vali = pd.read_csv(valiFilePath)
df_vali.head(10)

,RowID,BeerID,ReviewerID,rating,BeerType_Altbier,BeerType_AmericanAdjunctLager,BeerType_AmericanAmberRedAle,BeerType_AmericanAmberRedLager,BeerType_AmericanBarleywine,BeerType_AmericanBlackAle,...,Lemmatized_DocVec_199,ReviewerReviewCount,BeerReviewCount,ABV,DayofWeek,DayofMonth,Month,Year,TimeOfDay,Birthday
0,22,12300,2634,4.0,0,0,0,0,0,0,...,-0.062678,-0.794358,-0.841253,0.132531,1.011714,-0.194788,-0.453220,1.130340,-0.271244,-0.746074
1,27,12300,5634,4.5,0,0,0,0,0,0,...,-0.023707,-0.595446,-0.841253,0.132531,0.030280,-0.649105,-1.304288,1.130340,-0.283197,-0.746074
2,28,12300,3544,4.5,0,0,0,0,0,0,...,-0.011652,0.632318,-0.841253,0.132531,0.520997,-0.649105,1.532606,0.716016,1.153632,-0.746074
3,40,12300,6521,4.0,0,0,0,0,0,0,...,-0.030397,-0.369099,-0.841253,-0.696305,0.030280,1.281742,0.397849,0.301691,-0.366378,-0.746074
4,43,12300,10177,4.5,0,0,0,0,0,0,...,-0.031086,-0.526856,-0.841253,-0.696305,-1.441870,-0.649105,0.397849,0.301691,-1.010988,1.294013
5,48,12300,2907,3.5,0,0,0,0,0,0,...,-0.007063,0.652895,-0.841253,-0.696305,-0.460437,-0.081209,0.114159,0.301691,-0.409586,-0.746074
6,49,12300,1532,4.0,0,0,0,0,0,0,...,-0.022682,0.344239,-0.841253,-0.696305,-0.951153,-0.989843,0.114159,0.301691,-0.815949,1.160964
7,50,12300,3452,3.5,0,0,0,0,0,0,...,-0.013928,-0.670896,-0.841253,-0.696305,-1.441870,-1.103422,0.114159,0.301691,-0.357495,-0.746074
8,59,12300,6861,4.0,0,0,0,0,0,0,...,-0.014745,0.652895,-0.841253,-0.696305,1.011714,-1.330580,-0.736909,0.301691,0.215010,-0.746074
9,85539,1198,2634,4.5,0,0,0,0,0,0,...,-0.047180,-0.794358,3.631045,-1.176157,0.030280,1.508901,-1.587977,0.301691,-1.254554,0.761816


In [6]:
# "BrewerID_", "BeerType_", "ABV", "DayofWeek", "DayofMonth", "Month", "Year", "Gender_", "TimeOfDay", "Birthday", "BeerName_", "Lemmatized_"
# Get all the columns
col_names = df_train.columns

idCols = ['RowID','BeerID','ReviewerID']
# feature_cols =  col_names.drop(['RowID','BeerID','ReviewerID','rating' ])
target_col = 'rating'

feature_cols = []
# feature_cols.append("ReviewerReviewCount")
# feature_cols.append("BeerReviewCount")
#feature_cols = feature_cols + list(filter(lambda x: x.startswith("BeerName_"), col_names))
feature_cols = feature_cols + list(filter(lambda x: x.startswith("Lemmatized_"), col_names))

print(feature_cols)

['Lemmatized_DocVec_0', 'Lemmatized_DocVec_1', 'Lemmatized_DocVec_2', 'Lemmatized_DocVec_3', 'Lemmatized_DocVec_4', 'Lemmatized_DocVec_5', 'Lemmatized_DocVec_6', 'Lemmatized_DocVec_7', 'Lemmatized_DocVec_8', 'Lemmatized_DocVec_9', 'Lemmatized_DocVec_10', 'Lemmatized_DocVec_11', 'Lemmatized_DocVec_12', 'Lemmatized_DocVec_13', 'Lemmatized_DocVec_14', 'Lemmatized_DocVec_15', 'Lemmatized_DocVec_16', 'Lemmatized_DocVec_17', 'Lemmatized_DocVec_18', 'Lemmatized_DocVec_19', 'Lemmatized_DocVec_20', 'Lemmatized_DocVec_21', 'Lemmatized_DocVec_22', 'Lemmatized_DocVec_23', 'Lemmatized_DocVec_24', 'Lemmatized_DocVec_25', 'Lemmatized_DocVec_26', 'Lemmatized_DocVec_27', 'Lemmatized_DocVec_28', 'Lemmatized_DocVec_29', 'Lemmatized_DocVec_30', 'Lemmatized_DocVec_31', 'Lemmatized_DocVec_32', 'Lemmatized_DocVec_33', 'Lemmatized_DocVec_34', 'Lemmatized_DocVec_35', 'Lemmatized_DocVec_36', 'Lemmatized_DocVec_37', 'Lemmatized_DocVec_38', 'Lemmatized_DocVec_39', 'Lemmatized_DocVec_40', 'Lemmatized_DocVec_41', '

In [7]:

# Create the sub data sets of the features and the target
dfTrainIds = df_train[idCols]
dfTrainFeatures = df_train[feature_cols]
dfTrainTarget = df_train[target_col]

dfValiIds = df_vali[idCols]
dfValiFeatures = df_vali[feature_cols]
dfValiTarget = df_vali[target_col]



In [8]:
print(dfTrainFeatures.shape)
dfTrainFeatures.head()

(746207, 200)


,Lemmatized_DocVec_0,Lemmatized_DocVec_1,Lemmatized_DocVec_2,Lemmatized_DocVec_3,Lemmatized_DocVec_4,Lemmatized_DocVec_5,Lemmatized_DocVec_6,Lemmatized_DocVec_7,Lemmatized_DocVec_8,Lemmatized_DocVec_9,...,Lemmatized_DocVec_190,Lemmatized_DocVec_191,Lemmatized_DocVec_192,Lemmatized_DocVec_193,Lemmatized_DocVec_194,Lemmatized_DocVec_195,Lemmatized_DocVec_196,Lemmatized_DocVec_197,Lemmatized_DocVec_198,Lemmatized_DocVec_199
0,-0.022288,0.078109,-0.005654,-0.033343,0.036283,-0.037736,-0.039065,-0.023913,0.013213,0.079623,...,-0.015221,-0.037351,-0.027596,0.042438,0.032531,-0.028370,0.035328,0.010716,-0.016952,-0.022973
1,-0.053673,0.042035,0.047492,-0.046618,0.027993,-0.027121,-0.015928,-0.034591,0.028809,0.096345,...,-0.023733,-0.015190,-0.001721,0.023392,-0.019479,-0.005462,0.021144,0.027892,-0.010553,-0.038961
2,-0.032307,0.027868,0.042565,-0.037186,0.030322,-0.038627,-0.037316,-0.031006,0.022064,0.090013,...,-0.018624,-0.013615,-0.011761,0.032562,0.005066,-0.030226,0.019855,0.056897,-0.034139,-0.028913
3,-0.057763,0.031688,0.031182,-0.020936,0.024694,-0.033558,-0.053532,-0.029684,0.013137,0.097454,...,-0.032950,-0.008993,-0.009092,0.014841,-0.012791,-0.010687,0.020269,0.035415,-0.010456,-0.034556
4,-0.048823,0.047172,0.041055,-0.047864,0.029047,-0.040546,-0.025809,-0.034162,0.025955,0.088493,...,-0.005594,-0.021759,-0.006643,0.022896,-0.017797,-0.012587,0.017830,0.021566,-0.017301,-0.024277


In [9]:
# all cols best params
# {'learning_rate': 0.09075359977364383, 'num_leaves': 120, 'max_depth': 40, 'n_estimators ': 248, 'min_split_gain': 0.6310082232017945, 
# 'min_child_samples': 35, 'subsample': 0.9466694477903548, 
# 'subsample_freq': 0, 'colsample_bytree': 0.29392263338193186, 'reg_alpha': 0.891904482598078, 'reg_lambda': 0.4521335679885054}.

# 

# No tuning
# model = lgb.LGBMRegressor(objective="regression_l1", metric="mae", random_state=seed)

# Same settings as NLP Beer Name
model = lgb.LGBMRegressor(objective="regression_l1", metric="mae", random_state=seed
    ,learning_rate=0.2472480229316372, num_leaves = 125, max_depth = 24, n_estimators = 796
  )

model.fit(X=dfTrainFeatures, y=dfTrainTarget)

LGBMRegressor(learning_rate=0.2472480229316372, max_depth=24, metric='mae',
              n_estimators=796, num_leaves=125, objective='regression_l1',
              random_state=146433)

In [10]:
# Use the model to predict against our validation data
test_predicted = model.predict(dfValiFeatures)
test_predicted[0:100]

array([3.99980985, 4.0011835 , 3.8830903 , 4.00000456, 3.99584382,
       4.        , 3.95515178, 4.00122965, 4.        , 4.00000601,
       3.99996269, 4.        , 3.98636366, 4.        , 4.        ,
       4.00001007, 3.99967297, 3.9492059 , 4.        , 3.96961298,
       3.89106542, 3.89406116, 4.        , 4.07882562, 4.        ,
       4.01491036, 3.90485611, 4.        , 4.        , 3.80226908,
       4.        , 4.        , 3.99891459, 3.89305141, 3.90729631,
       3.99868188, 4.        , 3.89130854, 3.9910872 , 4.        ,
       4.04209235, 3.97477284, 4.        , 4.        , 4.        ,
       4.        , 3.96711632, 4.28300616, 3.87552011, 3.35377895,
       4.02577385, 4.        , 4.02369969, 3.95063432, 4.01144691,
       4.0108299 , 4.        , 3.76786664, 3.80978599, 4.00023941,
       4.        , 4.        , 3.77709922, 4.14739787, 3.98963879,
       4.12458666, 4.00000422, 4.        , 3.99981644, 4.00800562,
       4.00270698, 4.        , 3.87632888, 4.        , 4.     

In [11]:
# dfPredicted = pd.DataFrame({"Predict": test_predicted})
# dfPredicted['Predict'].hist(bins=10)


In [12]:
mae = mean_absolute_error(dfValiTarget, test_predicted)

print("Average MAE: " + str(mae))
print("analyse_maes.append(" + str(mae) + ")")

Average MAE: 0.5064167707585777
analyse_maes.append(0.5064167707585777)


In [13]:

# print("Run: " + filePrefix)
# # Log of Results
# analyse_maes = []

# print("Average MAE over all tests: " + str(np.mean(analyse_maes)))

# Make sure it's predicting floats
# dfPredicted["Predict"].drop_duplicates()

Write to a subrun file

In [14]:
import pandas
print("Average MAE: " + str(mae))

if writeSubRunFile:
  dfPredicted = pandas.DataFrame( { "RowID" : dfValiIds["RowID"], "Score": test_predicted } )
  dfPredicted.to_csv(subrunDir + filePrefix + "_subrun.csv", index=False)

  print(dfPredicted.shape)
  dfPredicted.sort_values("RowID").head(20)


Average MAE: 0.5064167707585777


# Summary

### Columns: Review Count + Beer Name
* Params: learning_rate=0.2472480229316372, num_leaves = 125, max_depth = 24, n_estimators = 796
* Average MAE: 0.4721882724938895


###  Columns: Just Beer Name
* Tested both Dask and Pandas, got the same MAE
* Params: learning_rate=0.2472480229316372, num_leaves = 125, max_depth = 24, n_estimators = 796
* Forgot to save MAE but it was around: 0.4524056055395564



###  Columns: Both NLPs Beer Name and Text Lemmatized
* Tested both Dask and Pandas, got the same MAE
* Params: learning_rate=0.2472480229316372, num_leaves = 125, max_depth = 24, n_estimators = 796
* Forgot to save MAE but it was around: 0.4524056055395564

###  Columns: Just Text Lemmatized
* 0.5064167707585777